# Example Seldon Core Deployments using Ksonnet


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

** If running on GCP then run following to add cluster-admin to your user account **

In [ ]:
!kubectl create clusterrolebinding my-cluster-admin-binding --clusterrole=cluster-admin --user=$(gcloud info --format="value(config.account)")

On most clusters you will probably need to add cluster-admin privledges

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [ ]:
!rm -rf my-ml-deployment && ks init my-ml-deployment 

In [ ]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=true --withAmbassador=true --singleNamespace=false --namespace=seldon --withRbac=true

In [ ]:
!cd my-ml-deployment && \
      ks apply default

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

Install gRPC modules for the prediction protos.

In [ ]:
!cp ../../proto/prediction.proto ./proto
!cd ../../proto/tensorflow && make create_protos
!cp -vr ../../proto/tensorflow/tensorflow .
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

## Create Model App

In [ ]:
!kubectl create namespace test1

In [ ]:
!rm -rf my-model && ks init my-model --namespace test1

In [ ]:
!cd my-model && \
    ks registry add seldon-core ../../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master

## Serve Single Model

In [ ]:
!cd my-model && \
    ks generate seldon-serve-simple-v1alpha2 mymodel --image seldonio/mock_classifier:1.0 --oauthKey=key --oauthSecret=secret && \
    ks apply default -c mymodel

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-025d03d -n test1

Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [ ]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' -n test1

### Get predictions

In [ ]:
import sys
sys.path.append("../../notebooks")
from seldon_utils import *
API_AMBASSADOR="localhost:8003"
API_GATEWAY_REST="localhost:8002"
API_GATEWAY_GRPC="localhost:8004"

#### REST Request

In [ ]:
r = rest_request_api_gateway("key","secret","test1",API_GATEWAY_REST)
print(r.text)

In [ ]:
r = rest_request_ambassador("mymodel","test1",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_api_gateway("key","secret","test1",rest_endpoint=API_GATEWAY_REST,grpc_endpoint=API_GATEWAY_GRPC)

In [ ]:
grpc_request_ambassador("mymodel","test1",API_AMBASSADOR)

In [ ]:
!cd my-model && \
    ks delete default -c mymodel && \
    ks component rm mymodel

## Serve AB Test

In [ ]:
!cd my-model && \
    ks generate seldon-abtest-v1alpha2 myabtest --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 --oauthKey=key --oauthSecret=secret && \
    ks apply default -c myabtest

In [ ]:
!kubectl rollout status deploy/myabtest-myabtest-41de5b8 -n test1
!kubectl rollout status deploy/myabtest-myabtest-df66c5c -n test1

In [ ]:
!kubectl get seldondeployments myabtest -o jsonpath='{.status}' -n test1

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
r = rest_request_api_gateway("key","secret","test1",API_GATEWAY_REST)
print(r.text)

In [ ]:
r = rest_request_ambassador("myabtest","test1",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_api_gateway("key","secret","test1",rest_endpoint=API_GATEWAY_REST,grpc_endpoint=API_GATEWAY_GRPC)

In [ ]:
grpc_request_ambassador("myabtest","test1",API_AMBASSADOR)

In [ ]:
!cd my-model && \
    ks delete default -c myabtest && \
    ks component rm myabtest

## Serve Multi-Armed Bandit

In [ ]:
!cd my-model && \
    ks generate seldon-mab-v1alpha2 mymab --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 --oauthKey=key --oauthSecret=secret && \
    ks apply default -c mymab

In [ ]:
!kubectl rollout status deploy/mymab-mymab-41de5b8 -n test1
!kubectl rollout status deploy/mymab-mymab-b8038b2 -n test1
!kubectl rollout status deploy/mymab-mymab-df66c5c -n test1

In [ ]:
!kubectl get seldondeployments mymab -o jsonpath='{.status}' -n test1

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
r = rest_request_api_gateway("key","secret","test1",API_GATEWAY_REST)
print(r.text)

In [ ]:
r = rest_request_ambassador("mymab","test1",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_api_gateway("key","secret","test1",rest_endpoint=API_GATEWAY_REST,grpc_endpoint=API_GATEWAY_GRPC)

In [ ]:
grpc_request_ambassador("mymab","test1",API_AMBASSADOR)

In [ ]:
!cd my-model && \
    ks delete default -c mymab && \
    ks component rm mymab

# Tear down

In [ ]:
!cd my-ml-deployment && ks delete default

In [ ]:
!rm -rf my-ml-deployment